In [63]:
import os
import yaml

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import seaborn as sns
import folium
from folium.plugins import HeatMap

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf

from copy import deepcopy
from datetime import datetime
import requests
import re


# Dataset Accidents

## Téléchargements des datasets 
- Lieux 
- Usagers
- Caractéristiques 
- Véhicules

Call API, Download csv dans le bon folder

In [2]:
# Call API Data Accidents
def get_datasets_url(url=None):
    url='https://www.data.gouv.fr/api/1/datasets/53698f4ca3a729239d2036df/'
    r = requests.get(url).json()
    return {el['title']: el['latest'] for el in r['resources'] if el['title'].endswith(".csv") and not el['title'].startswith("vehicules-immatricules") }

lieux_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("lieux")}
usagers_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("usagers")}
car_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("car")}
vehicule_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("vehicule")}

all_urls = [lieux_datasets, usagers_datasets, car_datasets,vehicule_datasets]

### download csvs if not already

for url_dict in all_urls:
    for file_name, url in url_dict.items():
        file_name = '../raw_data/' + file_name
        if not os.path.exists(file_name):
            response = requests.get(url)
            if response.status_code == 200:
                with open(file_name, 'wb') as f:
                    f.write(response.content)

### check datasets

folder = "../raw_data/"
os.listdir(folder)


['lieux_2017.csv',
 'vehicules_2018.csv',
 'usagers_2006.csv',
 'usagers_2012.csv',
 'usagers_2013.csv',
 'usagers_2007.csv',
 'vehicules_2019.csv',
 'lieux_2016.csv',
 'lieux_2014.csv',
 'usagers_2011.csv',
 'usagers_2005.csv',
 'usagers_2010.csv',
 'usagers.csv',
 'lieux_2015.csv',
 'lieux_2011.csv',
 'lieux_2005.csv',
 'vehicules_2022.csv',
 'usagers_2014.csv',
 '.DS_Store',
 'usagers_2015.csv',
 'lieux_2010.csv',
 'lieux_2006.csv',
 'lieux_2012.csv',
 'vehicules_2009.csv',
 'vehicules_2021.csv',
 'lieux.csv',
 'usagers_2017.csv',
 'caracteristiques-2017.csv',
 'Flux',
 'usagers_2016.csv',
 'vehicules_2020.csv',
 'vehicules_2008.csv',
 'lieux_2013.csv',
 'lieux_2007.csv',
 'lieux-2021.csv',
 'usagers-2018.csv',
 'caracteristiques_2018.csv',
 'caracteristiques_2019.csv',
 'usagers-2019.csv',
 'lieux-2020.csv',
 'lieux-2022.csv',
 'usagers-2022.csv',
 'caracteristiques_2022.csv',
 'vehi.csv',
 'vehicules-2017.csv',
 'lieux-2018.csv',
 'usagers-2021.csv',
 'caracteristiques_2009.csv',


#### Renommage des fichiers

In [3]:
chemin_fichier_yml = '../config.yml'
chemin_dossier = '../raw_data'


with open (chemin_fichier_yml, 'r') as f:
    config = yaml.safe_load(f)
    rename_config = config.get('rename')

for old_name, new_name in rename_config.items():
    chemin_ancien_fichier = os.path.join(chemin_dossier, old_name)
    chemin_nouveau_fichier = os.path.join(chemin_dossier, new_name)

    if os.path.exists(chemin_ancien_fichier):
        os.rename(chemin_ancien_fichier, chemin_nouveau_fichier)
        print(f"Fichier renommé : {old_name} -> {new_name}")
    else:
        print(f"Fichier non trouvé : {old_name}")


Fichier renommé : caracteristiques-2017.csv -> caracteristiques_2017.csv
Fichier renommé : caracteristiques-2018.csv -> caracteristiques_2018.csv
Fichier renommé : caracteristiques-2019.csv -> caracteristiques_2019.csv
Fichier renommé : caracteristiques-2020.csv -> caracteristiques_2020.csv
Fichier renommé : carcteristiques-2021.csv -> caracteristiques_2021.csv
Fichier renommé : carcteristiques-2022.csv -> caracteristiques_2022.csv
Fichier renommé : lieux-2017.csv -> lieux_2017.csv
Fichier renommé : lieux-2018.csv -> lieux_2018.csv
Fichier renommé : lieux-2019.csv -> lieux_2019.csv
Fichier renommé : lieux-2020.csv -> lieux_2020.csv
Fichier renommé : lieux-2021.csv -> lieux_2021.csv
Fichier renommé : lieux-2022.csv -> lieux_2022.csv
Fichier renommé : usagers-2017.csv -> usagers_2017.csv
Fichier renommé : usagers-2018.csv -> usagers_2018.csv
Fichier renommé : usagers-2019.csv -> usagers_2019.csv
Fichier renommé : usagers-2020.csv -> usagers_2020.csv
Fichier renommé : usagers-2021.csv -> 

#### Concaténation des 4 types de fichiers

In [3]:
# Passer par fonction et var env
def concat_files(starting_word):

    chemin_fichier_yml = '../config.yml'
    with open(chemin_fichier_yml, 'r') as f:
        config = yaml.safe_load(f)
        config_sep = config.get('sep')
        config_encoding = config.get('encoding')

    chemin_dossier = '../raw_data/'

    df_concat = pd.DataFrame()
    files = [file for file in os.listdir(chemin_dossier) if file.endswith('.csv') and file.startswith(starting_word)]

    print(files)
    for file in files:
        chemin_fichier = os.path.join(chemin_dossier, file)

        if file in config_sep:
            sep = config_sep[file]
        else:
            sep = ','

        if file in config_encoding:
            encoding = config_encoding[file]
        else:
            encoding = 'utf-8'

        df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding, dtype=str)

        df_concat = pd.concat([df_concat, df1])

    return df_concat

#utiliser yield ?


In [4]:
carac_df = concat_files("caracteristiques")
lieux_df = concat_files("lieux")
usager_df = concat_files("usagers")
vehi_df = concat_files("vehicules")


['caracteristiques_2018.csv', 'caracteristiques_2019.csv', 'caracteristiques_2022.csv', 'caracteristiques_2009.csv', 'caracteristiques_2021.csv', 'caracteristiques_2020.csv', 'caracteristiques_2008.csv', 'caracteristiques_2011.csv', 'caracteristiques_2005.csv', 'caracteristiques_2010.csv', 'caracteristiques_2006.csv', 'caracteristiques_2012.csv', 'caracteristiques_2013.csv', 'caracteristiques_2007.csv', 'caracteristiques_2017.csv', 'caracteristiques_2016.csv', 'caracteristiques_2014.csv', 'caracteristiques_2015.csv']


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_91694/28020598.py:29: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



['lieux_2017.csv', 'lieux_2016.csv', 'lieux_2014.csv', 'lieux_2015.csv', 'lieux_2011.csv', 'lieux_2005.csv', 'lieux_2010.csv', 'lieux_2006.csv', 'lieux_2012.csv', 'lieux.csv', 'lieux_2013.csv', 'lieux_2007.csv', 'lieux_2022.csv', 'lieux_2009.csv', 'lieux_2021.csv', 'lieux_2020.csv', 'lieux_2008.csv', 'lieux_2018.csv', 'lieux_2019.csv']
['usagers_2006.csv', 'usagers_2012.csv', 'usagers_2013.csv', 'usagers_2007.csv', 'usagers_2011.csv', 'usagers_2005.csv', 'usagers_2010.csv', 'usagers.csv', 'usagers_2014.csv', 'usagers_2015.csv', 'usagers_2017.csv', 'usagers_2016.csv', 'usagers_2018.csv', 'usagers_2019.csv', 'usagers_2021.csv', 'usagers_2009.csv', 'usagers_2008.csv', 'usagers_2020.csv', 'usagers_2022.csv']
['vehicules_2018.csv', 'vehicules_2019.csv', 'vehicules_2022.csv', 'vehicules_2009.csv', 'vehicules_2021.csv', 'vehicules_2020.csv', 'vehicules_2008.csv', 'vehicules_2005.csv', 'vehicules_2011.csv', 'vehicules_2010.csv', 'vehicules_2012.csv', 'vehicules_2006.csv', 'vehicules_2007.csv',

#### Export en .csv

In [6]:
vehi_df.to_csv('../raw_data/vehi.csv', index=False)
carac_df.to_csv('../raw_data/carac.csv', index=False)
lieux_df.to_csv('../raw_data/lieux.csv', index=False)
usager_df.to_csv('../raw_data/usagers.csv', index=False)


## Dataframe Vehicules


In [5]:
vehi_df.head(2)


,Num_Acc,senc,catv,occutc,obs,obsm,choc,manv,num_veh,id_vehicule,motor
0,201800000001,0,07,000,00,2,3,01,B01,NaN,NaN
1,201800000001,0,07,000,00,2,2,15,A01,NaN,NaN


Drop columns occutc, num_veh, motor, id_vehicule, senc


In [6]:
vehi_df_modif = vehi_df.drop(['occutc','motor', 'id_vehicule','senc'], axis=1, inplace=False)


Cleaning columns obs, obsm, choc, manv, 

In [7]:
"""Converting col obs in int"""
vehi_df_modif['obs'] = vehi_df_modif['obs'].fillna(-1).astype(float).astype(int)


In [8]:
"""Clean the column 'obs'"""
values_destribution_obs = vehi_df_modif.obs[vehi_df_modif.obs >= 0].value_counts(normalize=True)
# new_values_obs = np.random.choice(values_destribution_obs.index, size = vehi_df_modif['obs'].isna().sum(), p=values_destribution_obs.values)
# vehi_df_modif.loc[vehi_df_modif['obs'].isna(), 'obs'] = new_values_obs
new_values_obs2 = np.random.choice(values_destribution_obs.index, size = (vehi_df_modif['obs'] == -1).sum(), p=values_destribution_obs.values)
vehi_df_modif.loc[vehi_df_modif['obs'] == -1, 'obs'] = new_values_obs2


In [9]:
"""Converting col obsm in int"""
vehi_df_modif['obsm'] = vehi_df_modif['obsm'].fillna(-1).astype(float).astype(int)

"""Cleaning the column 'obsm'"""
values_destribution_obsm = vehi_df_modif.obsm[vehi_df_modif.obsm >= 0].value_counts(normalize=True)
# new_values_obsm = np.random.choice(values_destribution_obsm.index, size = vehi_df_modif['obsm'].isna().sum(), p=values_destribution_obsm.values)
# vehi_df_modif.loc[vehi_df_modif['obsm'].isna(), 'obsm'] = new_values_obsm
new_values_obsm2 = np.random.choice(values_destribution_obsm.index, size = (vehi_df_modif['obsm'] == -1).sum(), p=values_destribution_obsm.values)
vehi_df_modif.loc[vehi_df_modif['obsm'] == -1, 'obsm'] = new_values_obsm2


In [10]:
"""Converting col choc in int"""
vehi_df_modif['choc'] = vehi_df_modif['choc'].fillna(-1).astype(float).astype(int)

"""Clean the column 'choc'"""
values_destribution_choc = vehi_df_modif.choc[vehi_df_modif.choc >= 0].value_counts(normalize=True)
# new_values_choc = np.random.choice(values_destribution_choc.index, size = vehi_df_modif['choc'].isna().sum(), p=values_destribution_choc.values)
# vehi_df_modif.loc[vehi_df_modif['choc'].isna(), 'choc'] = new_values_choc
new_values_choc2 = np.random.choice(values_destribution_choc.index, size = (vehi_df_modif['choc'] == -1).sum(), p=values_destribution_choc.values)
vehi_df_modif.loc[vehi_df_modif['choc'] == -1, 'choc'] = new_values_choc2


In [11]:
"""Converting col manv in int"""
vehi_df_modif['manv'] = vehi_df_modif['manv'].fillna(0).astype(float).astype(int)

"""Clean the column 'manv'"""""
values_destribution_manv = vehi_df_modif.manv[vehi_df_modif.manv > 0].value_counts(normalize=True)
new_values_manv = np.random.choice(values_destribution_manv.index, size = vehi_df_modif['manv'].isna().sum(), p=values_destribution_manv.values)
vehi_df_modif.loc[vehi_df_modif['manv'].isna(), 'manv'] = new_values_manv
new_values_manv2 = np.random.choice(values_destribution_manv.index, size = (vehi_df_modif['manv'] == -1).sum(), p=values_destribution_manv.values)
vehi_df_modif.loc[vehi_df_modif['manv'] == -1, 'manv'] = new_values_manv2
new_values_manv3 = np.random.choice(values_destribution_manv.index, size = (vehi_df_modif['manv'] == 0).sum(), p=values_destribution_manv.values)
vehi_df_modif.loc[vehi_df_modif['manv'] == 0, 'manv'] = new_values_manv3


In [178]:
vehi_df_modif['catv'].value_counts(), vehi_df_modif['catv'].isna().sum()


(07     1016821
 7       220813
 33      144087
 10      107333
 02       87631
 30       77039
 01       67929
 32       41609
 31       40339
 05       21816
 34       21166
 1        20956
 14       19832
 15       17301
 17       15954
 2        14082
 37       12969
 99        8283
 13        7624
 04        7463
 03        5779
 50        5116
 21        4177
 38        3496
 36        2574
 40        2305
 3         2293
 20        2150
 18        2075
 43        1995
 80        1793
 0         1030
 16         890
 60         754
 35         527
 39         503
 09         300
 19         183
 08         120
 42          83
 12          79
 41          57
 06          39
 11          17
  -1         13
 Name: catv, dtype: int64,
 0)

In [179]:
# Remplacer '02' par '2', et espaces devant chiffres
vehi_df_modif['catv'] = vehi_df['catv'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)


In [182]:
vehi_df_modif['catv'] = vehi_df_modif['catv'].astype(int)

values_destribution_catv = vehi_df_modif['catv'][vehi_df_modif['catv'] > -1].value_counts(normalize=True)
new_values_catv = np.random.choice(values_destribution_catv.index, size = (vehi_df_modif['catv'] == -1).sum(), p=values_destribution_catv.values)
vehi_df_modif.loc[vehi_df_modif['catv'] == -1, 'catv'] = new_values_catv
# Remplacer les -1 par la distribution


In [183]:
vehi_df_modif['catv'].value_counts()


7     1237643
33     144087
10     107335
2      101713
1       88885
30      77041
32      41609
31      40339
5       21816
34      21166
14      19832
15      17301
17      15954
37      12969
99       8283
3        8072
13       7624
4        7463
50       5116
21       4177
38       3496
36       2574
40       2305
20       2150
18       2075
43       1995
80       1793
0        1030
16        890
60        754
35        527
39        503
9         300
19        183
8         120
42         83
12         79
41         57
6          39
11         17
Name: catv, dtype: int64

In [191]:
vehi_df_modif.head(2)


,Num_Acc,catv,obs,obsm,choc,manv,num_veh
0,201800000001,7,0,2,3,1,B01
1,201800000001,7,0,2,2,15,A01


Export en .csv

In [192]:
vehi_df_modif.to_csv('../clean_data/vehicules_clean.csv', index=False)


## Dataset Caracteristiques

In [15]:
carac_df.head(2)


,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,Accident_Id
0,201800000001,18,1,24,1505,1,1,4,1,1,005,route des Ansereuilles,M,5055737,0294992,590,NaN
1,201800000002,18,2,12,1015,1,2,7,7,7,011,Place du général de Gaul,M,5052936,0293151,590,NaN


Drop des col gps et accidents_id

In [16]:
carac_df_modif = carac_df.drop(['gps', 'Accident_Id'], axis=1, inplace=False)


Formatage des 4 col date en une (optionnel)

In [ ]:
# def process_dates(df):
#     # Vérifier les dates invalides
#     invalid_dates = df[(df['jour'] > 31) | ((df['mois'] == 2) & (df['jour'] > 29)) | ((df['mois'].isin([4, 6, 9, 11])) & (df['jour'] > 30))]
#     df = df.drop(invalid_dates.index)

#     # Créer la colonne 'date'
#     df['date'] = pd.to_datetime(df['an'].astype(str).str.zfill(2)
#                                 + df['mois'].astype(str).str.zfill(2)
#                                 + df['jour'].astype(str).str.zfill(2)
#                                 + df['hrmn'].astype(str).str.zfill(4),
#                                 format='%y%m%d%H%M', errors='coerce')

#     df = df.drop(columns=['an', 'mois', 'jour', 'hrmn'])
#     return df

# # Utiliser la fonction sur carac_df_modif
# carac_df_modif = process_dates(carac_df_modif)


Cleaning des col Num_ACC, com, lum, int, col, atm

In [17]:
"""Drop Na de Num_Acc, com"""
carac_df_modif = carac_df_modif.dropna(subset=['Num_Acc','com'])


In [18]:
"""Converting col lum as int"""
carac_df_modif['lum'] = carac_df_modif['lum'].astype(int)

"""Clean the column 'lum'"""
values_destribution_lum = carac_df_modif.lum[carac_df_modif['lum'] >= 0].value_counts(normalize=True)
new_values_lum = np.random.choice(values_destribution_lum.index, size = (carac_df_modif['lum'] == -1).sum(), p=values_destribution_lum.values)
carac_df_modif.loc[carac_df_modif['lum'] == -1, 'lum'] = new_values_lum


In [19]:
"""Converting col int as int"""
carac_df_modif['int'] = carac_df_modif['int'].astype(int)

"""Delete wrong values column int"""
carac_df_modif = carac_df_modif.loc[carac_df_modif['int'] != -1]


In [20]:
"""Converting col col as int"""
carac_df_modif = carac_df_modif.dropna(subset=['col'])
carac_df_modif['col'] = carac_df_modif['col'].astype(int)

"""Clean the column 'col'"""
most_frequent = carac_df_modif['col'].mode()[0]
carac_df_modif['col'] = carac_df_modif['col'].replace(-1, most_frequent)


In [21]:
"""Converting col atm as int"""
carac_df_modif = carac_df_modif.dropna(subset=['atm'])
carac_df_modif['atm'] = carac_df_modif['atm'].astype(int)

"""Clean the column 'atm'"""
most_frequent = carac_df_modif['atm'].mode()[0]
carac_df_modif['atm'] = carac_df_modif['atm'].replace(-1, most_frequent)


In [22]:
carac_df_modif.isna().sum()


Num_Acc         0
an              0
mois            0
jour            0
hrmn            0
lum             0
agg             0
int             0
atm             0
col             0
com             0
adr        143270
lat        487017
long       487021
dep             0
dtype: int64

In [23]:
carac_df_modif.head(2)


,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,lat,long,dep
0,201800000001,18,1,24,1505,1,1,4,1,1,005,route des Ansereuilles,5055737,0294992,590
1,201800000002,18,2,12,1015,1,2,7,7,7,011,Place du général de Gaul,5052936,0293151,590


Export en .csv

In [24]:
carac_df_modif.to_csv('../clean_data/caracteristiques_clean.csv', index=False)


## Dataset Lieux

In [25]:
lieux_df.head(2)


,Num_Acc,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1,vma
0,201700000001,3,39,NaN,NaN,2,02,NaN,NaN,2,1,1,000,070,1,0,1,99,NaN
1,201700000002,3,39,NaN,NaN,2,02,NaN,NaN,0,1,0,006,010,1,NaN,NaN,NaN,NaN


Cleaning col circ, nbv, vosp, prof, pr, plan, lartpc, larrout, surf, infra, situ, catr

In [26]:
lieux_df['circ'].value_counts(), lieux_df['circ'].isna().sum()


(2       2225469
 2.0     1483646
 1        626649
 3        467319
 1.0      417766
 3.0      311546
 0        149898
 0.0       99932
  -1       36552
 -1.0      24368
 4         20010
 4.0       13340
 Name: circ, dtype: int64,
 7870)

In [27]:
lieux_df['circ'] = lieux_df['circ'].fillna(2, inplace=False)


In [28]:
# Conversion des valeurs de type '2.0' en '2' tout en restant en str
lieux_df['circ'] = lieux_df['circ'].astype(str)
lieux_df['circ'] = lieux_df['circ'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)


In [29]:
lieux_df['circ'] = lieux_df['circ'].astype(int)


In [30]:
#Remplacer les -1 par la distrib
values_destribution_circ = lieux_df['circ'][lieux_df['circ'] > -1].value_counts(normalize=True)
new_values_manv = np.random.choice(values_destribution_circ.index, size = (lieux_df['circ'] == -1).sum(), p=values_destribution_circ.values)
lieux_df.loc[lieux_df['circ'] == -1, 'circ'] = new_values_manv


In [31]:
#Remplacer les 0 par la distrib
values_destribution_circ = lieux_df['circ'][lieux_df['circ'] > 0].value_counts(normalize=True)
new_values_manv2 = np.random.choice(values_destribution_circ.index, size = (lieux_df['circ'] == 0).sum(), p=values_destribution_circ.values)
lieux_df.loc[lieux_df['circ'] == 0, 'circ'] = new_values_manv2


In [32]:
lieux_df['circ'].value_counts()


2    3923977
1    1102858
3     822274
4      35256
Name: circ, dtype: int64

In [33]:
lieux_df['nbv'].value_counts(), lieux_df['nbv'].isna().sum()


(02      1607787
 2.0      981182
 2        756901
 01       345204
 00       326451
          ...   
 41.0          1
 33.0          1
 52.0          1
 28.0          1
 86.0          1
 Name: nbv, Length: 118, dtype: int64,
 13655)

In [34]:
lieux_df['nbv'] = lieux_df['nbv'].fillna(2, inplace=False)


In [35]:
# Conversion des valeurs de type '2.0' en '2' tout en restant en str
lieux_df['nbv'] = lieux_df['nbv'].astype(str)
lieux_df['nbv'] = lieux_df['nbv'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
lieux_df['nbv'] = lieux_df['nbv'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)


In [36]:
lieux_df['nbv'].value_counts()


2          3359525
1           687510
4           576740
0           570635
3           457170
6           112185
5            64330
8            24445
-1           11150
7             8575
10            5110
9             2655
20            1205
50             790
11             720
12             710
40             150
30             135
13              90
25              70
21              55
26              45
15              40
90              35
24              30
14              30
22              25
70              25
31              15
60              15
16              10
45              10
17              10
27              10
53              10
41               5
33               5
#ERREUR          5
52               5
44               5
28               5
91               5
86               5
54               5
76               5
99               5
42               5
65               5
36               5
18               5
84               5
39               5
29          

In [37]:
lieux_df['nbv'] = pd.to_numeric(lieux_df['nbv'], errors='coerce')
lieux_df['nbv'] = lieux_df['nbv'].where(lieux_df['nbv'] <= 4, 2)
lieux_df['nbv'] = lieux_df['nbv'].replace({-1: 2})
# on remplace les valeurs abbérantes(>10) et les valeurs nulles par la valeur la plus fréquente


In [38]:
lieux_df['nbv'] = lieux_df['circ'].astype(int)


In [39]:
lieux_df['nbv'].value_counts()


2    3923977
1    1102858
3     822274
4      35256
Name: nbv, dtype: int64

In [40]:
lieux_df['vosp'].value_counts(), lieux_df['vosp'].isna().sum()


(0       3272235
 0.0     2181490
 3        103191
 1         89385
 3.0       68794
 1.0       59590
 2         53559
 2.0       35706
  -1        3966
 -1.0       2644
 Name: vosp, dtype: int64,
 13805)

In [41]:
lieux_df['vosp'] = lieux_df['vosp'].fillna(0, inplace=False)


In [42]:
# Conversion des valeurs de type '2.0' en '2' tout en restant en str
lieux_df['vosp'] = lieux_df['vosp'].astype(str)
lieux_df['vosp'] = lieux_df['vosp'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
lieux_df['vosp'] = lieux_df['vosp'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)


In [43]:
lieux_df['vosp'] = pd.to_numeric(lieux_df['vosp'], errors='coerce')
lieux_df['vosp'] = lieux_df['vosp'].replace({-1: 0})
# on remplace les valeurs nulles (1k) par la plus fréquente

lieux_df['vosp'].value_counts()


0    5474140
3     171985
1     148975
2      89265
Name: vosp, dtype: int64

In [44]:
lieux_df['prof'].value_counts(), lieux_df['prof'].isna().sum()


(1       2712174
 1.0     1808116
 2        504351
 2.0      336234
 0        197838
 0.0      131892
 3         62421
 4         47874
 3.0       41614
 4.0       31916
  -1         114
 -1.0         76
 Name: prof, dtype: int64,
 9745)

In [45]:
lieux_df['prof'] = lieux_df['prof'].fillna(1, inplace=False)

# Conversion des valeurs de type '2.0' en '2' tout en restant en str
lieux_df['prof'] = lieux_df['prof'].astype(str)
lieux_df['prof'] = lieux_df['prof'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
lieux_df['prof'] = lieux_df['prof'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)

lieux_df['prof'] = pd.to_numeric(lieux_df['prof'], errors='coerce')
lieux_df['prof'] = lieux_df['prof'].replace({-1: 1,0: 1})
# on remplace les valeurs nulles (65k) par la plus fréquente


In [46]:
lieux_df['prof'].value_counts()


1    4859955
2     840585
3     104035
4      79790
Name: prof, dtype: int64

In [47]:
lieux_df['pr'].value_counts(), lieux_df['pr'].isna().sum()


(0         794386
 0.0       300074
 (1)       270365
 1         172713
 2          74913
            ...  
 2760.0         2
 1930.0         2
 3892.0         2
 1580.0         2
 1006.0         2
 Name: pr, Length: 1653, dtype: int64,
 2377420)

In [48]:
lieux_df['pr1'].value_counts(), lieux_df['pr1'].isna().sum()
# On drop la colonne pr et la pr1 qui ne sera pas plus utile


(0         972328
 0.0       396052
 (1)       274095
 500       125664
 200        74003
            ...  
 2526.0         2
 1674.0         2
 1668.0         2
 2680.0         2
 3640.0         2
 Name: pr1, Length: 4635, dtype: int64,
 2386515)

In [49]:
lieux_df['plan'].value_counts(), lieux_df['plan'].isna().sum()


(1       2710068
 1.0     1806712
 2        298236
 3        270771
 0        199251
 2.0      198824
 3.0      180514
 0.0      132834
 4         45348
 4.0       30232
  -1         102
 -1.0         68
 Name: plan, dtype: int64,
 11405)

In [50]:
lieux_df['plan'] = lieux_df['plan'].fillna(1, inplace=False)

# Conversion des valeurs de type '2.0' en '2' tout en restant en str
lieux_df['plan'] = lieux_df['plan'].astype(str)
lieux_df['plan'] = lieux_df['plan'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
lieux_df['plan'] = lieux_df['plan'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)

lieux_df['plan'] = pd.to_numeric(lieux_df['plan'], errors='coerce')
lieux_df['plan'] = lieux_df['plan'].replace({-1: 1,0: 1})
# on remplace les valeurs nulles (65k) par la plus fréquente


In [51]:
lieux_df['plan'].value_counts()


1    4860440
2     497060
3     451285
4      75580
Name: plan, dtype: int64

In [52]:
lieux_df['lartpc'].value_counts(), lieux_df['lartpc'].isna().sum()
#30% de valeurs nulles on drop la colonne


(000      2276046
 0.0      1354417
 0         164687
 015       102069
 15.0       61850
           ...   
 780.0          1
 295.0          1
 910.0          1
 570.0          1
 680.0          1
 Name: lartpc, Length: 974, dtype: int64,
 1370530)

In [53]:
pd.set_option('display.max_rows', None)
lieux_df['larrout'].value_counts(), lieux_df['larrout'].isna().sum()
# on drop, trop valeurs aberrantes


(000                    863124
  -1                    747400
 0.0                    513018
 060                    228585
 070                    183276
 60.0                   133620
 70.0                   106689
 080                     83205
 050                     75354
 120                     75169
 100                     68651
 0                       63633
 140                     63434
 090                     55278
 105                     52407
 80.0                    48176
 030                     44808
 50.0                    43735
 120.0                   40411
 040                     39573
 100.0                   36204
 035                     35919
 140.0                   34031
 065                     33483
 075                     32673
 90.0                    31896
 105.0                   28748
 30.0                    26427
 072                     25455
 40.0                    23073
 062                     22383
 210                     22233
 35.0   

In [54]:
lieux_df['surf'] = lieux_df['surf'].fillna(1, inplace=False)

# Conversion des valeurs de type '2.0' en '2' tout en restant en str
lieux_df['surf'] = lieux_df['surf'].astype(str)
lieux_df['surf'] = lieux_df['surf'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
lieux_df['surf'] = lieux_df['surf'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)

lieux_df['surf'] = pd.to_numeric(lieux_df['surf'], errors='coerce')
lieux_df['surf'] = lieux_df['surf'].replace({-1: 1,0: 1, 9: 1})
# on remplace les valeurs nulles (65k) par la plus fréquente


In [55]:
lieux_df['surf'].value_counts()


1    4794010
2    1010755
7      34740
5      16425
8      13675
3       8355
6       3505
4       2900
Name: surf, dtype: int64

In [56]:
lieux_df['infra'].value_counts(), lieux_df['infra'].isna().sum()


(0       3096267
 0.0     2064178
 5        205572
 5.0      137048
 3         53304
 2         51849
 3.0       35536
 2.0       34566
 1         32262
 6         24987
 9         24111
 1.0       21508
 6.0       16658
 9.0       16074
 4         12339
 4.0        8226
  -1        6708
 8          4899
 -1.0       4472
 8.0        3266
 7          2097
 7.0        1398
 Name: infra, dtype: int64,
 27040)

In [57]:
lieux_df['infra'] = lieux_df['infra'].fillna(1, inplace=False)

# Conversion des valeurs de type '2.0' en '2' tout en restant en str
lieux_df['infra'] = lieux_df['infra'].astype(str)
lieux_df['infra'] = lieux_df['infra'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
lieux_df['infra'] = lieux_df['infra'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)

lieux_df['infra'] = pd.to_numeric(lieux_df['infra'], errors='coerce')
lieux_df['infra'] = lieux_df['infra'].replace({-1: 1})
# on remplace les valeurs nulles par la plus fréquente,
# 0 ne l'étant pas car ça veut dire qu'il n'y a pas d'infra,
# 9 peut être pertinent car c'est une infra 'autre'


In [58]:
lieux_df['infra'].value_counts()


0    5160445
5     342620
1      91990
3      88840
2      86415
6      41645
9      40185
4      20565
8       8165
7       3495
Name: infra, dtype: int64

In [59]:
lieux_df['situ'].value_counts(), lieux_df['situ'].isna().sum()


(1       2949954
 1.0     1966636
 3        276807
 3.0      184538
 0        142374
 0.0       94916
 4         62058
 4.0       41372
 5         31272
 2         25368
 5.0       20848
 8         19710
 2.0       16912
 8.0       13140
 6          7740
 6.0        5160
  -1         426
 -1.0        284
 Name: situ, dtype: int64,
 24850)

In [60]:
lieux_df['situ'] = lieux_df['situ'].fillna(1, inplace=False)

# Conversion des valeurs de type '2.0' en '2' tout en restant en str
lieux_df['situ'] = lieux_df['situ'].astype(str)
lieux_df['situ'] = lieux_df['situ'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
lieux_df['situ'] = lieux_df['situ'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)

lieux_df['situ'] = pd.to_numeric(lieux_df['situ'], errors='coerce')
lieux_df['situ'] = lieux_df['situ'].replace({-1: 1, 0: 1})
# on remplace les valeurs nulles par la plus fréquentes


In [61]:
lieux_df['situ'].value_counts()


1    5179440
3     461345
4     103430
5      52120
2      42280
8      32850
6      12900
Name: situ, dtype: int64

In [62]:
lieux_df['catr'].value_counts(), lieux_df['catr'].isna().sum()


(4      1715616
 3      1157427
 4.0    1143744
 3.0     771618
 1       279267
 2       269340
 1.0     186178
 2.0     179560
 9        56964
 9.0      37976
 6        23946
 7        21825
 6.0      15964
 7.0      14550
 5         6231
 5.0       4154
 Name: catr, dtype: int64,
 5)

In [63]:
lieux_df['catr'] = lieux_df['catr'].fillna(4, inplace=False)

# Conversion des valeurs de type '2.0' en '2' tout en restant en str
lieux_df['catr'] = lieux_df['catr'].astype(str)
lieux_df['catr'] = lieux_df['catr'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
lieux_df['catr'] = lieux_df['catr'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)


In [64]:
lieux_df['catr'].value_counts()


4    2859365
3    1929045
1     465445
2     448900
9      94940
6      39910
7      36375
5      10385
Name: catr, dtype: int64

Drop des col

In [65]:
lieux_df_clean = lieux_df.drop(columns=['voie', 'v1', 'v2', 'pr', 'pr1', 'lartpc', 'larrout', 'vma', 'env1'])


In [66]:
lieux_df_clean.head(2)


,Num_Acc,catr,circ,nbv,vosp,prof,plan,surf,infra,situ
0,201700000001,3,2,2,2,1,1,1,0,1
1,201700000002,3,2,2,0,1,1,1,1,1


Export .csv

In [67]:
lieux_df_clean.to_csv('../clean_data/lieux_clean.csv', index=False)


## Dataset usagers

In [68]:
usager_df.head(2)


,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh,id_vehicule,secu1,secu2,secu3,id_usager
0,200600000001,1,1,4,1,5,11,0,0,0,1949,B01,NaN,NaN,NaN,NaN,NaN
1,200600000001,2,2,4,2,0,11,0,0,0,1948,B01,NaN,NaN,NaN,NaN,NaN


Cleaning

In [69]:
usager_df['grav'].value_counts(), usager_df['grav'].isna().sum()


(1      5413730
 4      4809425
 3      2604085
 2       353140
  -1        903
 -1         602
 Name: grav, dtype: int64,
 0)

In [70]:
usager_df['grav'] = usager_df['grav'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)
# Remplacer '02' par '2', et espaces devant chiffres

usager_df['grav'] = pd.to_numeric(usager_df['grav'], errors='coerce')
usager_df['grav'] = usager_df['grav'].replace({-1: 1})
# on remplace les valeurs nulle par la plus fréquente


In [71]:
usager_df['grav'].value_counts()


1    5415235
4    4809425
3    2604085
2     353140
Name: grav, dtype: int64

In [72]:
usager_df['sexe'].value_counts(), usager_df['sexe'].isna().sum()


(1      8865950
 2      4286905
  -1      17418
 -1       11612
 Name: sexe, dtype: int64,
 0)

In [73]:
usager_df['sexe'] = usager_df['sexe'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)
# Remplacer '02' par '2', et espaces devant chiffres

usager_df['sexe'] = pd.to_numeric(usager_df['sexe'], errors='coerce')
usager_df['sexe'] = usager_df['sexe'].replace({-1: 1})
# on remplace les valeurs nulle par la plus fréquente


In [74]:
usager_df['sexe'].value_counts()


1    8894980
2    4286905
Name: sexe, dtype: int64

In [75]:
usager_df['trajet'].value_counts(), usager_df['trajet'].isna().sum()


(5       2935245
 0       2204202
 5.0     1956830
 0.0     1469468
 1       1034712
 4        767256
 1.0      689808
 9        568125
 4.0      511504
 9.0      378750
 3        213120
 2        164289
 3.0      142080
 2.0      109526
  -1       20700
 -1.0      13800
 Name: trajet, dtype: int64,
 2470)

In [76]:
usager_df['trajet'] = usager_df['trajet'].fillna(5, inplace=False)

# Conversion des valeurs de type '2.0' en '2' tout en restant en str
usager_df['trajet'] = usager_df['trajet'].astype(str)
usager_df['trajet'] = usager_df['trajet'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
usager_df['trajet'] = usager_df['trajet'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)

usager_df['trajet'] = pd.to_numeric(usager_df['trajet'], errors='coerce')
usager_df['trajet'] = usager_df['trajet'].replace({-1: 5})
# Remplacez les valeurs -1 par 5


In [77]:
usager_df['trajet'] = usager_df['trajet'].astype(int)

values_destribution_trajet = usager_df['trajet'][usager_df['trajet'] > 0].value_counts(normalize=True)
new_values_manv = np.random.choice(values_destribution_trajet.index, size = (usager_df['trajet'] == 0).sum(), p=values_destribution_trajet.values)
usager_df.loc[usager_df['trajet'] == 0, 'trajet'] = new_values_manv
#Remplacer les 0 par la distrib


In [78]:
usager_df['trajet'].value_counts()


5    6833702
1    2391418
4    1772901
9    1312371
3     492075
2     379418
Name: trajet, dtype: int64

In [79]:
usager_df['secu'].value_counts(), usager_df['secu'].isna().sum()


(11      3592401
 11.0    2394934
 21      1168512
 21.0     779008
 13       500739
 13.0     333826
 93       315363
 93.0     210242
 00       205122
 0.0      136748
 23       122595
 12       108945
 23.0      81730
 12.0      72630
 22        67122
 31        44928
 22.0      44748
 31.0      29952
 3         23121
 92        23079
 91        22959
 10        16893
 3.0       15414
 92.0      15386
 91.0      15306
 10.0      11262
 1         10701
 2          8007
 41         7596
 1.0        7136
 43         6012
 42         5568
 2.0        5338
 41.0       5064
 43.0       4008
 42.0       3712
 33         2511
 32         2283
 20         2262
 33.0       1674
 32.0       1522
 20.0       1508
 90          219
 90.0        146
 30           27
 30.0         18
 40            6
 40.0          4
 01            3
 Name: secu, dtype: int64,
 2753595)

In [80]:
usager_df['secu'] = usager_df['secu'].fillna(usager_df['secu1'])
#on remplace les valeurs nulles par les valeurs nulles de secu (avant 2019) par les valeurs de secu1(après 2019)


In [81]:
usager_df['secu'] = usager_df['secu'].fillna(11, inplace=False)


In [82]:
# Conversion des valeurs de type '2.0' en '2' tout en restant en str
usager_df['secu'] = usager_df['secu'].astype(str)
usager_df['secu'] = usager_df['secu'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
usager_df['secu'] = usager_df['secu'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)


In [83]:
usager_df['secu'] = pd.to_numeric(usager_df['secu'], errors='coerce')
usager_df['secu'] = usager_df['secu'].apply(lambda x: 1 if x > 0 else 0)
# on normalise les valeurs secu (0 = Pas d'équipement, 1 = Equipement)


In [84]:
usager_df['secu'].value_counts()


1    12606150
0      575735
Name: secu, dtype: int64

In [85]:
usager_df['locp'].value_counts(), usager_df['locp'].isna().sum()


(0       6487995
 0.0     4325330
  -1      650259
 -1.0     433506
 2        151401
 4        147990
 3        138810
 2.0      100934
 4.0       98660
 1         93897
 3.0       92540
 1.0       62598
 5         42213
 5.0       28142
 6         14109
 6.0        9406
 8          7101
 9          5532
 8.0        4734
 9.0        3688
 7           741
 7.0         494
 Name: locp, dtype: int64,
 281805)

In [86]:
usager_df['locp'] = usager_df['locp'].fillna(0, inplace=False)
# Remplacer les valeurs manquantes dans 'locp' par 0


In [87]:
# Conversion des valeurs de type '2.0' en '2' tout en restant en str
usager_df['locp'] = usager_df['locp'].astype(str)
usager_df['locp'] = usager_df['locp'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
usager_df['locp'] = usager_df['locp'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)


In [88]:
# Remplacer les valeurs -1 par 0 dans 'locp'
usager_df['locp'] = pd.to_numeric(usager_df['locp'], errors='coerce')
usager_df['locp'] = usager_df['locp'].replace({-1: 0})


In [89]:
usager_df['locp'].value_counts(), usager_df['locp'].isna().sum()


(0    12178895
 2      252335
 4      246650
 3      231350
 1      156495
 5       70355
 6       23515
 8       11835
 9        9220
 7        1235
 Name: locp, dtype: int64,
 0)

In [90]:
usager_df['actp'].value_counts(), usager_df['actp'].isna().sum()
# On drop cette colonne, la répertition est trop compliquée


(0      7382879
 0.0    3536496
  -1     928330
 3       552184
 3.0     244811
 9        51046
 1        42992
 5        40465
 9.0      22709
 2        21083
 5.0      19200
 1.0      18148
 4        12867
 2.0       8987
 B         6170
 4.0       5968
 A         2110
 6         1811
 6.0        759
 7          300
 8          260
 Name: actp, dtype: int64,
 282310)

In [91]:
usager_df['etatp'].value_counts(), usager_df['etatp'].isna().sum()


(0       5744379
 0.0     3829586
  -1     1368738
 -1.0     912492
 1        475443
 1.0      316962
 2        124527
 2.0       83018
 3         26790
 3.0       17860
 Name: etatp, dtype: int64,
 282090)

In [92]:
usager_df['etatp'] = usager_df['etatp'].fillna(0, inplace=False)


In [93]:
# Conversion des valeurs de type '2.0' en '2' tout en restant en str
usager_df['etatp'] = usager_df['etatp'].astype(str)
usager_df['etatp'] = usager_df['etatp'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
usager_df['etatp'] = usager_df['etatp'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)


In [94]:
usager_df['etatp'].value_counts()
# On drop la colonne, trop de -1 et trop de 0 qui correspondent à l'absence de valeur


0     9856055
-1    2281230
1      792405
2      207545
3       44650
Name: etatp, dtype: int64

In [95]:
usager_df['an_nais'].value_counts(), usager_df['an_nais'].isna().sum()


(1988      198846
 1987      198723
 1986      198591
 1989      193491
 1985      192525
 1990      188949
 1984      180072
 1982      172533
 1991      172431
 1983      170316
 1980      166017
 1981      165537
 1992      159888
 1979      145554
 1993      142491
 1978      139749
 1972      138459
 1977      138036
 1973      137988
 1971      135276
 1974      134235
 1994      133440
 1988.0    132564
 1987.0    132482
 1986.0    132394
 1975      132246
 1976      131139
 1970      129951
 1989.0    128994
 1985.0    128350
 1995      126249
 1990.0    125966
 1969      124311
 1968      121962
 1966      120726
 1984.0    120048
 1967      119730
 1965      119124
 1964      118293
 1963      115371
 1996      115326
 1982.0    115022
 1991.0    114954
 1983.0    113544
 1980.0    110678
 1981.0    110358
 1962      108621
 1961      107103
 1992.0    106592
 1997      106218
 1960      103704
 1998       99033
 1959       98688
 1979.0     97036
 1993.0     94994
 1978.0   

In [96]:
# Drop les lignes avec des valeurs manquantes dans 'an_nais'
usager_df = usager_df.dropna(subset=['an_nais'], inplace=False)


In [97]:
# Conversion des valeurs de type '2.0' en '2' tout en restant en str
usager_df['an_nais'] = usager_df['an_nais'].astype(str)
usager_df['an_nais'] = usager_df['an_nais'].apply(lambda x: str(int(float(x))) if x.replace('.', '', 1).lstrip('-').isdigit() and '.' in x else x)

# Remplacer '02' par '2', et espaces devant chiffres
usager_df['an_nais'] = usager_df['an_nais'].apply(lambda x: str(int(x.strip())) if x.strip().replace('-', '').isdigit() else x)


In [98]:
print(usager_df[~usager_df['an_nais'].str.match(r'^\d{4}$')])
# Checking des valeur différentes de 'YYYY'


Empty DataFrame
Columns: [Num_Acc, place, catu, grav, sexe, trajet, secu, locp, actp, etatp, an_nais, num_veh, id_vehicule, secu1, secu2, secu3, id_usager]
Index: []


In [99]:
usager_df['an_nais'].value_counts()


1988    331410
1987    331205
1986    330985
1989    322485
1985    320875
1990    314915
1984    300120
1982    287555
1991    287385
1983    283860
1980    276695
1981    275895
1992    266480
1979    242590
1993    237485
1978    232915
1972    230765
1977    230060
1973    229980
1971    225460
1974    223725
1994    222400
1975    220410
1976    218565
1970    216585
1995    210415
1969    207185
1968    203270
1966    201210
1967    199550
1965    198540
1964    197155
1963    192285
1996    192210
1962    181035
1961    178505
1997    177030
1960    172840
1998    165055
1959    164480
1958    153505
1999    152465
1957    148670
2000    146720
1956    141070
1955    135830
1954    129365
2001    125805
1953    120475
1952    117645
1950    112110
1951    109635
2002    106095
1949    102405
1948     98445
1947     94475
1946     87365
2003     87270
2004     70915
1945     69045
1944     65800
1943     64060
2005     59740
1942     57240
2006     50685
1941     50465
1940     5

Drop de colonnes

In [100]:
usager_df_clean = usager_df.drop(columns=['place','catu', 'secu1','secu2','secu3','id_vehicule','id_usager', 'etatp', 'actp'])
# on drop les colonnes non pertinentes


In [101]:
usager_df_clean.isna().sum()


Num_Acc    0
grav       0
sexe       0
trajet     0
secu       0
locp       0
an_nais    0
num_veh    0
dtype: int64

In [102]:
usager_df_clean.head(2)


,Num_Acc,grav,sexe,trajet,secu,locp,an_nais,num_veh
0,200600000001,4,1,5,1,0,1949,B01
1,200600000001,4,2,5,1,0,1948,B01


Export .csv

In [103]:
usager_df_clean.to_csv('../clean_data/usagers_clean.csv', index=False)
# on charge le fichier nettoyé


## Merging des 4 Datasets

In [2]:
lieux_df = pd.read_csv('../clean_data/lieux_clean.csv', dtype=str)
usagers_df = pd.read_csv('../clean_data/usagers_clean.csv', dtype=str)
vehicules_df = pd.read_csv('../clean_data/vehicules_clean.csv', dtype=str)
caracteristiques_df = pd.read_csv('../clean_data/caracteristiques_clean.csv', dtype=str)


In [3]:
lieux_df.head(2), lieux_df.shape


(        Num_Acc catr circ nbv vosp prof plan surf infra situ
 0  201700000001    3    2   2    2    1    1    1     0    1
 1  201700000002    3    2   2    0    1    1    1     1    1,
 (5884365, 10))

In [4]:
caracteristiques_df.head(2), caracteristiques_df.shape


(        Num_Acc  an mois jour  hrmn lum agg int atm col  com  \
 0  201800000001  18    1   24  1505   1   1   4   1   1  005   
 1  201800000002  18    2   12  1015   1   2   7   7   7  011   
 
                         adr      lat     long  dep  
 0    route des Ansereuilles  5055737  0294992  590  
 1  Place du général de Gaul  5052936  0293151  590  ,
 (1121482, 15))

In [5]:
vehicules_df.head(2), vehicules_df.shape


(        Num_Acc catv obs obsm choc manv num_veh
 0  201800000001    7   0    2    3    1     B01
 1  201800000001    7   0    2    2   15     A01,
 (2009395, 7))

In [6]:
usagers_df.head(2), usagers_df.shape


(        Num_Acc grav sexe trajet secu locp an_nais num_veh
 0  200600000001    4    1      5    1    0    1949     B01
 1  200600000001    4    2      5    1    0    1948     B01,
 (13140090, 8))

In [7]:
filtered_df = vehicules_df[vehicules_df['catv'] == '1']


In [8]:
filtered_df.head(2), filtered_df.shape


(         Num_Acc catv obs obsm choc manv num_veh
 6   201800000004    1   0    2    8    1     B01
 30  201800000020    1  12    0    7    1     B01,
 (88885, 7))

In [9]:
duplicates = filtered_df[filtered_df['Num_Acc'].duplicated(keep=False)]
print(duplicates.shape)


(5399, 7)


In [10]:
caracteristiques_df.isna().sum()


Num_Acc         0
an              0
mois            0
jour            0
hrmn            0
lum             0
agg             0
int             0
atm             0
col             0
com             0
adr        143270
lat        487017
long       487021
dep             0
dtype: int64

In [11]:
a = caracteristiques_df[caracteristiques_df['Num_Acc'].isna()]
# fonction rolliwn window ? applique une fonction sur une partie précise


In [12]:
# Création d'une colonne uniques pour merger les df usagers et véhicules
usagers_df['aug'] = usagers_df['Num_Acc'].astype(str) + usagers_df['num_veh'].astype(str)
vehicules_df['aug'] = vehicules_df['Num_Acc'].astype(str) + vehicules_df['num_veh'].astype(str)


In [13]:
vehicules_df = vehicules_df[vehicules_df['catv']=='1']
# On drop les duplicates lorsqu'il y a une plusieurs usagers dans un même véhicules et pour le même accident
# On crée une colonne pour y ajouter le nombre d'usagers par véhicules


In [14]:
usagers_df['dup_count'] = usagers_df.groupby('aug')['aug'].transform('count')
usagers_df =usagers_df.drop_duplicates(subset=['aug'], keep='first')
print(type(vehicules_df))
print(type(usagers_df))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [15]:
# On merge véhicules et usagers puis on ne garde que les accidents impliquant un vélo
# On clean et on drop les doublons

vehi_usa = pd.merge(vehicules_df, usagers_df, on='aug', how='inner')
vehi_usa = vehi_usa.drop(columns=['num_veh_x', 'num_veh_y', 'Num_Acc_y'])
vehi_usa = vehi_usa.rename(columns={'Num_Acc_x': 'Num_Acc'})

vehi_usa['dup_count'].value_counts()


5     83223
10     5132
15      156
20        5
30        3
25        2
Name: dup_count, dtype: int64

In [16]:
print(vehi_usa['Num_Acc'].duplicated().any())
print(caracteristiques_df['Num_Acc'].duplicated().any())


True
False


In [17]:
duplicates_velo = vehi_usa[vehi_usa['Num_Acc'].duplicated(keep=False)]
duplicates_velo.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5148 entries, 12 to 88515
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Num_Acc    5148 non-null   object
 1   catv       5148 non-null   object
 2   obs        5148 non-null   object
 3   obsm       5148 non-null   object
 4   choc       5148 non-null   object
 5   manv       5148 non-null   object
 6   aug        5148 non-null   object
 7   grav       5148 non-null   object
 8   sexe       5148 non-null   object
 9   trajet     5148 non-null   object
 10  secu       5148 non-null   object
 11  locp       5148 non-null   object
 12  an_nais    5148 non-null   object
 13  dup_count  5148 non-null   int64 
dtypes: int64(1), object(13)
memory usage: 603.3+ KB


In [18]:
# On merge les df caractéristiques et lieux
velo_df = pd.merge(vehi_usa, lieux_df, on='Num_Acc', how='left')
velo_df = pd.merge(velo_df, caracteristiques_df, on='Num_Acc', how='left')

velo_df.info()
velo_df.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 442605 entries, 0 to 442604
Data columns (total 37 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Num_Acc    442605 non-null  object
 1   catv       442605 non-null  object
 2   obs        442605 non-null  object
 3   obsm       442605 non-null  object
 4   choc       442605 non-null  object
 5   manv       442605 non-null  object
 6   aug        442605 non-null  object
 7   grav       442605 non-null  object
 8   sexe       442605 non-null  object
 9   trajet     442605 non-null  object
 10  secu       442605 non-null  object
 11  locp       442605 non-null  object
 12  an_nais    442605 non-null  object
 13  dup_count  442605 non-null  int64 
 14  catr       442605 non-null  object
 15  circ       442605 non-null  object
 16  nbv        442605 non-null  object
 17  vosp       442605 non-null  object
 18  prof       442605 non-null  object
 19  plan       442605 non-null  object
 20  surf

,Num_Acc,catv,obs,obsm,choc,manv,aug,grav,sexe,trajet,...,lum,agg,int,atm,col,com,adr,lat,long,dep
0,201800000004,1,0,2,8,1,201800000004B01,3,1,5,...,1,2,1,7,3,052,30 rue Jules Guesde,5051974,0289123,590
1,201800000004,1,0,2,8,1,201800000004B01,3,1,5,...,1,2,1,7,3,052,30 rue Jules Guesde,5051974,0289123,590
2,201800000004,1,0,2,8,1,201800000004B01,3,1,5,...,1,2,1,7,3,052,30 rue Jules Guesde,5051974,0289123,590
3,201800000004,1,0,2,8,1,201800000004B01,3,1,5,...,1,2,1,7,3,052,30 rue Jules Guesde,5051974,0289123,590
4,201800000004,1,0,2,8,1,201800000004B01,3,1,5,...,1,2,1,7,3,052,30 rue Jules Guesde,5051974,0289123,590


In [19]:
# On remplace les valeurs manquantes par la valeur la plus fréquente
velo_df['lum'].fillna(velo_df['lum'].mode()[0], inplace=True)
velo_df['agg'].fillna(velo_df['agg'].mode()[0], inplace=True)
velo_df['atm'].fillna(velo_df['atm'].mode()[0], inplace=True)
velo_df['atm'] = velo_df['atm'].replace({ -1: 1, 9: 1})


In [20]:
# Obtenez la distribution des valeurs
distribution = velo_df['int'].value_counts(normalize=True)
distribution


1    0.571631
2    0.151181
3    0.132809
6    0.073055
4    0.020835
9    0.020547
5    0.014395
7    0.014251
8    0.001274
0    0.000024
Name: int, dtype: float64

In [21]:
# Obtenez les valeurs manquantes
missing = velo_df['int'].isnull()


In [22]:
# Remplissez les valeurs manquantes en fonction de la distribution
velo_df.loc[missing, 'int'] = np.random.choice(distribution.index, size=len(velo_df[missing]), p=distribution.values)


In [23]:
# Obtenez la distribution des valeurs
distribution = velo_df['col'].value_counts(normalize=True)
distribution


3    0.487257
6    0.181039
1    0.118642
2    0.115758
7    0.063166
5    0.025497
4    0.008639
Name: col, dtype: float64

In [24]:
# Obtenez les valeurs manquantes
missing = velo_df['col'].isnull()


In [25]:
# Remplissez les valeurs manquantes en fonction de la distribution
velo_df.loc[missing, 'col'] = np.random.choice(distribution.index, size=len(velo_df[missing]), p=distribution.values)


In [26]:
# Je remplace le -1 par 1
velo_df['col'] = velo_df['col'].replace({ -1: 3})
velo_df.info()
velo_df = velo_df[~(velo_df.adr.isna() & velo_df.lat.isna() & velo_df.long.isna() & velo_df.com.isna() & velo_df.dep.isna())]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 442605 entries, 0 to 442604
Data columns (total 37 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Num_Acc    442605 non-null  object
 1   catv       442605 non-null  object
 2   obs        442605 non-null  object
 3   obsm       442605 non-null  object
 4   choc       442605 non-null  object
 5   manv       442605 non-null  object
 6   aug        442605 non-null  object
 7   grav       442605 non-null  object
 8   sexe       442605 non-null  object
 9   trajet     442605 non-null  object
 10  secu       442605 non-null  object
 11  locp       442605 non-null  object
 12  an_nais    442605 non-null  object
 13  dup_count  442605 non-null  int64 
 14  catr       442605 non-null  object
 15  circ       442605 non-null  object
 16  nbv        442605 non-null  object
 17  vosp       442605 non-null  object
 18  prof       442605 non-null  object
 19  plan       442605 non-null  object
 20  surf

In [27]:
# Réinitialisez l'index de votre DataFrame
velo_df = velo_df.reset_index(drop=True)


In [28]:
velo_df.columns


Index(['Num_Acc', 'catv', 'obs', 'obsm', 'choc', 'manv', 'aug', 'grav', 'sexe',
       'trajet', 'secu', 'locp', 'an_nais', 'dup_count', 'catr', 'circ', 'nbv',
       'vosp', 'prof', 'plan', 'surf', 'infra', 'situ', 'an', 'mois', 'jour',
       'hrmn', 'lum', 'agg', 'int', 'atm', 'col', 'com', 'adr', 'lat', 'long',
       'dep'],
      dtype='object')

In [29]:
# Obtenez les valeurs manquantes
velo_df['an'].isnull().sum(), velo_df['mois'].isnull().sum(), velo_df['jour'].isnull().sum(), velo_df['hrmn'].isnull().sum()


(0, 0, 0, 0)

In [30]:
velo_df.drop(columns=['catv'], inplace=True)


In [31]:
velo_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416125 entries, 0 to 416124
Data columns (total 36 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Num_Acc    416125 non-null  object
 1   obs        416125 non-null  object
 2   obsm       416125 non-null  object
 3   choc       416125 non-null  object
 4   manv       416125 non-null  object
 5   aug        416125 non-null  object
 6   grav       416125 non-null  object
 7   sexe       416125 non-null  object
 8   trajet     416125 non-null  object
 9   secu       416125 non-null  object
 10  locp       416125 non-null  object
 11  an_nais    416125 non-null  object
 12  dup_count  416125 non-null  int64 
 13  catr       416125 non-null  object
 14  circ       416125 non-null  object
 15  nbv        416125 non-null  object
 16  vosp       416125 non-null  object
 17  prof       416125 non-null  object
 18  plan       416125 non-null  object
 19  surf       416125 non-null  object
 20  infr

In [32]:
print(velo_df['com'])


0         052
1         052
2         052
3         052
4         052
         ... 
416120    611
416121    611
416122    611
416123    611
416124    611
Name: com, Length: 416125, dtype: object


In [33]:
velo_df.head(2), velo_df.shape


(        Num_Acc obs obsm choc manv              aug grav sexe trajet secu  \
 0  201800000004   0    2    8    1  201800000004B01    3    1      5    1   
 1  201800000004   0    2    8    1  201800000004B01    3    1      5    1   
 
    ... lum agg  int atm col  com                  adr      lat     long  dep  
 0  ...   1   2    1   7   3  052  30 rue Jules Guesde  5051974  0289123  590  
 1  ...   1   2    1   7   3  052  30 rue Jules Guesde  5051974  0289123  590  
 
 [2 rows x 36 columns],
 (416125, 36))

Export .csv dataframe mergé

In [34]:
velo_df.to_csv('../clean_data/velo_df.csv', index=False)


### Reconstitution lat, long pour Paris

##### Call API

In [85]:
data = pd.read_csv('../clean_data/velo_df.csv', dtype=str)


In [86]:
data_filtered_paris = data.loc[data['dep'].isin(['750'])]


In [87]:
data_filtered_paris.head()


,Num_Acc,obs,obsm,choc,manv,aug,grav,sexe,trajet,secu,...,lum,agg,int,atm,col,com,adr,lat,long,dep
20235,201800050213,0,2,3,7,201800050213B01,4,1,5,1,...,1,2,2,2,6,101,"12, QUAI FRANCOIS MITTER",4886010,0233333,750
20236,201800050213,0,2,3,7,201800050213B01,4,1,5,1,...,1,2,2,2,6,101,"12, QUAI FRANCOIS MITTER",4886010,0233333,750
20237,201800050213,0,2,3,7,201800050213B01,4,1,5,1,...,1,2,2,2,6,101,"12, QUAI FRANCOIS MITTER",4886010,0233333,750
20238,201800050213,0,2,3,7,201800050213B01,4,1,5,1,...,1,2,2,2,6,101,"12, QUAI FRANCOIS MITTER",4886010,0233333,750
20239,201800050213,0,2,3,7,201800050213B01,4,1,5,1,...,1,2,2,2,6,101,"12, QUAI FRANCOIS MITTER",4886010,0233333,750


In [88]:
data_filtered_paris.shape


(51050, 36)

In [89]:
data_filtered_paris.isna().sum()


Num_Acc          0
obs              0
obsm             0
choc             0
manv             0
aug              0
grav             0
sexe             0
trajet           0
secu             0
locp             0
an_nais          0
dup_count        0
catr             0
circ             0
nbv              0
vosp             0
prof             0
plan             0
surf             0
infra            0
situ             0
an               0
mois             0
jour             0
hrmn             0
lum              0
agg              0
int              0
atm              0
col              0
com              0
adr              0
lat          39370
long         39370
dep              0
dtype: int64

In [90]:
def filter_missing(data, col1, col2):
    """
    Filters a DataFrame to keep only rows where both specified columns have missing values.

    Parameters:
    data (pd.DataFrame): The DataFrame to be filtered.
    col1
    col2

    Returns:
    pd.DataFrame: A DataFrame containing rows where both specified columns have missing values.
    """
    filtered_data = data[data[col1].isna() | data[col2].isna()]
    return filtered_data


In [91]:
data_filtered_missinglatlong = filter_missing(data_filtered_paris, 'lat', 'long')


In [92]:
data_filtered_missinglatlong.shape


(39370, 36)

In [105]:
import re

def remove_comma_period_slash_start(data, column):
    """
    Removes commas, periods, and slashes from the start of each string in the specified column of a DataFrame.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the column to be modified.
    column (str): The name of the column to modify.

    Returns:
    pd.DataFrame: The DataFrame with the specified column modified.
    """

    def clean_string(x):
        # This function uses regex to remove commas, periods, and slashes from the start of the string
        if isinstance(x, str):
            return re.sub(r'^[,.\\/]+', '', x)
        return x

    data[column] = data[column].apply(clean_string)
    return data


In [109]:
remove_comma_period_slash_start(data_filtered_missinglatlong, 'adr')


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_92683/1580107784.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Num_Acc,obs,obsm,choc,manv,aug,grav,sexe,trajet,secu,...,lum,agg,int,atm,col,com,adr,lat,long,dep
20340,201800050313,0,2,8,1,201800050313B01,4,1,5,1,...,5,2,2,1,3,109,RUE LA FAYETTE,NaN,NaN,750
20341,201800050313,0,2,8,1,201800050313B01,4,1,5,1,...,5,2,2,1,3,109,RUE LA FAYETTE,NaN,NaN,750
20342,201800050313,0,2,8,1,201800050313B01,4,1,5,1,...,5,2,2,1,3,109,RUE LA FAYETTE,NaN,NaN,750
20343,201800050313,0,2,8,1,201800050313B01,4,1,5,1,...,5,2,2,1,3,109,RUE LA FAYETTE,NaN,NaN,750
20344,201800050313,0,2,8,1,201800050313B01,4,1,5,1,...,5,2,2,1,3,109,RUE LA FAYETTE,NaN,NaN,750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415285,201500056190,0,2,3,15,201500056190A01,4,2,5,1,...,1,2,4,1,3,118,"1, AVENUE DE LA PORTE DE",NaN,NaN,750
415286,201500056190,0,2,3,15,201500056190A01,4,2,5,1,...,1,2,4,1,3,118,"1, AVENUE DE LA PORTE DE",NaN,NaN,750
415287,201500056190,0,2,3,15,201500056190A01,4,2,5,1,...,1,2,4,1,3,118,"1, AVENUE DE LA PORTE DE",NaN,NaN,750
415288,201500056190,0,2,3,15,201500056190A01,4,2,5,1,...,1,2,4,1,3,118,"1, AVENUE DE LA PORTE DE",NaN,NaN,750


In [110]:
filtered_values = data_filtered_missinglatlong[data_filtered_missinglatlong['adr'].str.startswith('/')]
count = len(filtered_values)
print(count)


0


In [111]:
data_filtered_missinglatlong.head()


,Num_Acc,obs,obsm,choc,manv,aug,grav,sexe,trajet,secu,...,lum,agg,int,atm,col,com,adr,lat,long,dep
20340,201800050313,0,2,8,1,201800050313B01,4,1,5,1,...,5,2,2,1,3,109,RUE LA FAYETTE,NaN,NaN,750
20341,201800050313,0,2,8,1,201800050313B01,4,1,5,1,...,5,2,2,1,3,109,RUE LA FAYETTE,NaN,NaN,750
20342,201800050313,0,2,8,1,201800050313B01,4,1,5,1,...,5,2,2,1,3,109,RUE LA FAYETTE,NaN,NaN,750
20343,201800050313,0,2,8,1,201800050313B01,4,1,5,1,...,5,2,2,1,3,109,RUE LA FAYETTE,NaN,NaN,750
20344,201800050313,0,2,8,1,201800050313B01,4,1,5,1,...,5,2,2,1,3,109,RUE LA FAYETTE,NaN,NaN,750


In [112]:
# def get_lat_lon(address, url='https://api-adresse.data.gouv.fr/search'):

#     params = {'q': address, 'citycode': 75056}
#     r = requests.get(url, params)
#     try: return r.json()['features'][0]['geometry']['coordinates'][::-1]
#     except: print(f'ERROR : {address}, STATUS CODE : {r.status_code}')


In [116]:
request_count = 0
def get_lat_lon(address, url='https://api-adresse.data.gouv.fr/search'):
    global request_count
    request_count += 1  # Incrémenter le compteur à chaque appel

    params = {'q': address, 'citycode': 75056}
    r = requests.get(url, params)

    try:
        coords = r.json()['features'][0]['geometry']['coordinates'][::-1]
    except:
        print(f'ERROR : {address}, STATUS CODE : {r.status_code}')
        coords = None

    # Vérifier si le compteur a atteint un multiple de 500
    if request_count % 500 == 0:
        print(f'500 requêtes ont été effectuées. Total actuel : {request_count}')

    return coords


In [117]:
data_filtered_missinglatlong['latlong'] = data_filtered_missinglatlong['adr'].map(lambda address: get_lat_lon(address))


500 requêtes ont été effectuées. Total actuel : 500
500 requêtes ont été effectuées. Total actuel : 1000
500 requêtes ont été effectuées. Total actuel : 1500
500 requêtes ont été effectuées. Total actuel : 2000
500 requêtes ont été effectuées. Total actuel : 2500
500 requêtes ont été effectuées. Total actuel : 3000
ERROR : AVENUE CESAR CAIRE, STATUS CODE : 504
500 requêtes ont été effectuées. Total actuel : 3500
500 requêtes ont été effectuées. Total actuel : 4000
ERROR : 16, RUE DE LA FONTAINE A, STATUS CODE : 504
500 requêtes ont été effectuées. Total actuel : 4500
ERROR : SOUTERRAIN GENERAL LEMON, STATUS CODE : 200
ERROR : SOUTERRAIN GENERAL LEMON, STATUS CODE : 200
ERROR : SOUTERRAIN GENERAL LEMON, STATUS CODE : 200
ERROR : SOUTERRAIN GENERAL LEMON, STATUS CODE : 200
ERROR : SOUTERRAIN GENERAL LEMON, STATUS CODE : 200
500 requêtes ont été effectuées. Total actuel : 5000
500 requêtes ont été effectuées. Total actuel : 5500
500 requêtes ont été effectuées. Total actuel : 6000
500 req

/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_92683/2777418506.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [118]:
data_filtered_missinglatlong['latlong'].isna().sum()


491

In [120]:
data_filtered_missinglatlong.to_csv('../clean_data/velo_df_aftercall.csv', index=False)


In [121]:
data_filtered_missinglatlong['latlong'] = data_filtered_missinglatlong['latlong'].astype(str)

data_filtered_missinglatlong['latlong'] = data_filtered_missinglatlong['latlong'].str.strip('[]').str.replace(' ', '')

data_filtered_missinglatlong[['lat', 'long']] = data_filtered_missinglatlong['latlong'].str.split(',', expand=True)


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_92683/3373048785.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_92683/3373048785.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_92683/3373048785.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [128]:
data_filtered_missinglatlong.head(2)


,Num_Acc,obs,obsm,choc,manv,aug,grav,sexe,trajet,secu,...,agg,int,atm,col,com,adr,lat,long,dep,latlong
20340,201800050313,0,2,8,1,201800050313B01,4,1,5,1,...,2,2,1,3,109,RUE LA FAYETTE,48.879907,2.35947,750,"48.879907,2.35947"
20341,201800050313,0,2,8,1,201800050313B01,4,1,5,1,...,2,2,1,3,109,RUE LA FAYETTE,48.879907,2.35947,750,"48.879907,2.35947"


In [129]:
data_filtered_missinglatlong = data_filtered_missinglatlong.drop(columns=['latlong'], inplace=False)


In [131]:
data_filtered_missinglatlong.dropna(subset=['lat', 'long'], inplace=True)


In [133]:
data_filtered_missinglatlong.isna().sum()


Num_Acc      0
obs          0
obsm         0
choc         0
manv         0
aug          0
grav         0
sexe         0
trajet       0
secu         0
locp         0
an_nais      0
dup_count    0
catr         0
circ         0
nbv          0
vosp         0
prof         0
plan         0
surf         0
infra        0
situ         0
an           0
mois         0
jour         0
hrmn         0
lum          0
agg          0
int          0
atm          0
col          0
com          0
adr          0
lat          0
long         0
dep          0
dtype: int64

##### Formatage lat long


In [ ]:
"""Formatage non nécessaire ici, le format est bon"""

# def reformatter_lat_lon(dataframe):
#     """
#     Reformate les colonnes de latitude et de longitude d'une DataFrame Pandas,
#     en remplaçant les virgules par des points et en assurant que les valeurs sont dans un format décimal correct.
#     """

#     if 'lat' in dataframe.columns and 'long' in dataframe.columns:
#         for col in ['lat', 'long']:
#             dataframe[col] = pd.to_numeric(dataframe[col].astype(str).str.replace(',', '.'), errors='ignore')
#     else:
#         raise ValueError("La DataFrame doit contenir les colonnes 'lat' et 'long'")

#     return dataframe


In [136]:
data_filtered_missinglatlong.to_csv('../clean_data/velo_df_paris.csv', index=False)


# Dataset Pistes

In [ ]:
def get_datasets_url(url):
    response = requests.get(url).json()
    return {el['title']: el['latest'] for el in response['resources'] if el['title'].endswith(".csv") and not el['title'].startswith("vehicules-immatricules")}


def download_and_save_datasets(url_dict, save_path):
    for path, url in url_dict.items():
        full_path = os.path.join(save_path, path)
        if not os.path.exists(full_path):
            response = requests.get(url)
            if response.status_code == 200:
                with open(full_path, 'wb') as f:
                    f.write(response.content)


def rename_files(config_path, save_path):
    with open(config_path, 'r') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
        rename_config = config.get('rename')

    for old_name, new_name in rename_config.items():
        old_file_path = os.path.join(save_path, old_name)
        new_file_path = os.path.join(save_path, new_name)

        if os.path.exists(old_file_path):
            os.rename(old_file_path, new_file_path)
            print(f"Fichier renommé : {old_name} -> {new_name}")
        else:
            print(f"Fichier non trouvé : {old_name}")


In [ ]:
url = 'https://transport.data.gouv.fr/api/datasets/60a37b7f303fdf4f2654b73d'


In [ ]:
"""Drop colonnes non-pertinentes"""
data_pistes_clean = data_pistes.drop(['num_iti', 'largeur_d', 'local_d', 'largeur_g', 'local_g', 'access_ame', 'statut_d', 'd_service'], axis=1, inplace=False)


In [ ]:
"""Drop Na colonne code_com_d, regime_d, sens_d, ame_d, code_com_g"""
data_pistes_clean = data_pistes_clean.dropna(subset=['code_com_d', 'regime_d', 'sens_d', 'ame_d', 'code_com_g'])


In [ ]:
"""Cleaning colonne lumiere"""
values_distribution_lumiere = data_pistes_clean.lumiere.value_counts(normalize=True)
new_values_lumiere = np.random.choice(values_distribution_lumiere.index, size = data_pistes_clean['lumiere'].isna().sum(), p=values_distribution_lumiere.values)
data_pistes_clean.loc[data_pistes_clean.lumiere.isna(), 'lumiere'] = new_values_lumiere


In [ ]:
"""Cleaning colonne revet_d"""
values_distribution_revet = data_pistes_clean.revet_d.value_counts(normalize=True)
new_values_revet = np.random.choice(values_distribution_revet.index, size = data_pistes_clean['revet_d'].isna().sum(), p=values_distribution_revet.values)
data_pistes_clean.loc[data_pistes_clean.revet_d.isna(), 'revet_d'] = new_values_revet


In [ ]:
"""Cleaning colonne revet_g"""
values_distribution_revetg = data_pistes_clean.revet_g.value_counts(normalize=True)
new_values_revetg = np.random.choice(values_distribution_revetg.index, size = data_pistes_clean['revet_g'].isna().sum(), p=values_distribution_revetg.values)
data_pistes_clean.loc[data_pistes_clean.revet_g.isna(), 'revet_g'] = new_values_revetg


In [ ]:
"""Cleaning colonne regime_g"""
values_distribution_regimeg = data_pistes_clean.regime_g.value_counts(normalize=True)
new_values_regimeg = np.random.choice(values_distribution_regimeg.index, size = data_pistes_clean['regime_g'].isna().sum(), p=values_distribution_regimeg.values)
data_pistes_clean.loc[data_pistes_clean.regime_g.isna(), 'regime_g'] = new_values_regimeg


In [ ]:
"""Cleaning colonne sens_g"""
values_distribution_sensg = data_pistes_clean.sens_g.value_counts(normalize=True)
new_values_sensg = np.random.choice(values_distribution_sensg.index, size = data_pistes_clean['sens_g'].isna().sum(), p=values_distribution_sensg.values)
data_pistes_clean.loc[data_pistes_clean.sens_g.isna(), 'sens_g'] = new_values_sensg


In [ ]:
"""Cleaning colonne trafic_vit"""
values_distribution_trafic = data_pistes_clean.trafic_vit.value_counts(normalize=True)
new_values_trafic = np.random.choice(values_distribution_trafic.index, size = data_pistes_clean['trafic_vit'].isna().sum(), p=values_distribution_trafic.values)
data_pistes_clean.loc[data_pistes_clean.trafic_vit.isna(), 'trafic_vit'] = new_values_trafic


# Data Flux

In [ ]:
fichier_compteurs = '/Users/aurelienbrame/code/CyclingFacilities/raw_data/Flux/comptage-velo-compteurs.geojson'
fichier_compteurs_data = '/Users/aurelienbrame/code/CyclingFacilities/raw_data/Flux/comptage-velo-donnees-compteurs.geojson'


In [ ]:
data_flux_compteurs = gpd.read_file(fichier_compteurs)
data_flux_compteurs_data = gpd.read_file(fichier_compteurs_data)


In [ ]:
data_flux_compteurs_data.head()


In [ ]:
data_flux_compteurs_data.columns


In [ ]:
data_flux_compteurs_data.drop(['url_photos_n1', 'type_dimage', 'photos', 'id_photo_1',
                               'test_lien_vers_photos_du_site_de_comptage_', 'url_sites',
                               'coordinates', 'name', 'id'],
                              axis=1, inplace=False)


In [ ]:
data_flux_months = pd.to_datetime(data_flux_compteurs_data['date'])
counts_by_month = data_flux_compteurs_data.groupby(data_flux_compteurs_data['date'].dt.to_period('M')).sum()
counts_by_month
